<a href="https://colab.research.google.com/github/Mxrkxzzz/Basic-Chatbot/blob/main/Chat_de_Voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Librería transformers para manejar datos como el LPN.
!pip install transformers

In [ ]:
## librerías colab_utils y librosa, permite cargar y visualizar archivos de audio dentro de su propia aplicación
!pip install git+https://github.com/ricardodeazambuja/colab_utils.git
!pip install librosa==0.8.1

In [ ]:
## librerías para el procesamiento del audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from colab_utils import getAudio
import librosa
## librería para creación de vectores y matrices.
import numpy as np
## pytorch para la conversión de voz a texto
import torch

In [ ]:
w2v2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
w2v2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
## método getAudio de la librería colab_utils.. para capturar el audio desde nuestro micrófono.
audio, sr = getAudio()


In [ ]:
# Representar cada muestra en formato punto flotante con 32 bits
audio_float = audio.astype(np.float32)

# Y cambiar la frecuencia de muestreo a 16 KHz
audio_16k = librosa.resample(audio_float, sr, 16000)


In [ ]:
# conversión de voz a texto
entrada = w2v2_processor(audio_16k, sampling_rate=16000, return_tensors="pt").input_values

In [ ]:
## arreglo de 203 x 32
probabilidades = w2v2(entrada).logits

In [ ]:
predicciones = torch.argmax(probabilidades, dim=-1)

In [ ]:
## pasa de una representación numérica a una secuencia de caracteres.
transcripcion = w2v2_processor.decode(predicciones[0])

In [ ]:
## librería que se utiliza para procesar el texto a un formato que sea entendible para el modelo.
## tokenizador separará el texto en palabras llamadas tokens
## tokens, consiste en la segmentación del texto en frases o palabras
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")


In [ ]:
entradaBlender = tokenizer([transcripcion], return_tensors='pt')

In [ ]:
from transformers import AutoModelForSeq2SeqLM
blender = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

In [ ]:
ids_respuesta = blender.generate(**entradaBlender)

In [ ]:
## tensor, ## Son una generalización de vectores y matrices y se entienden fácilmente como una matriz multidimensional
print(ids_respuesta)

In [ ]:
respuesta = tokenizer.batch_decode(ids_respuesta)

In [ ]:
print(respuesta)

In [ ]:
respuesta = respuesta[0].replace('<s>','').replace('</s>','')

In [ ]:
NFRASES = 6
nfrase = 1

while nfrase <= NFRASES:
  input()     # Esperar a pulsar tecla para iniciar grabación

  # Capturar audio y llevarlo a 16 KHz
  audio, sr = getAudio()
  audio_float = audio.astype(np.float32)
  audio_16k = librosa.resample(audio_float, sr, 16000)

  # Voz a texto con wav2vec2
  entrada = w2v2_processor(audio_16k, sampling_rate=16000, return_tensors="pt").input_values
  probabilidades = w2v2(entrada).logits
  predicciones = torch.argmax(probabilidades, dim=-1)
  frase = w2v2_processor.decode(predicciones[0])

  # Imprimir transcripción voz a texto
  print(f'-> MIGUEL: {frase}')

  # BlenderBot
  entradaBlender = tokenizer([frase], return_tensors='pt')
  ids_respuesta = blender.generate(**entradaBlender)
  respuesta = tokenizer.batch_decode(ids_respuesta)
  respuesta = respuesta[0].replace('<s>','').replace('</s>','')
  print(f'-> BLENDERBOT: {respuesta}')

  nfrase += 1